In [2]:
import os
import csv
import json
import math
import torch
import argparse
import difflib
import logging
import numpy as np
import pandas as pd

from transformers import BertTokenizer, BertForMaskedLM
from transformers import AlbertTokenizer, AlbertForMaskedLM
from transformers import RobertaTokenizer, RobertaForMaskedLM
from collections import defaultdict
from tqdm import tqdm

In [57]:
def read_data(input_file):
    """
    Load data into pandas DataFrame format.
    """
    
    df_data = pd.DataFrame(columns=['sent1', 'sent2', 'direction', 'bias_type'])

    list_data = []

    with open(input_file, encoding='utf-8-sig') as f:
        reader = csv.DictReader(f)
        for row in reader:
            direction, gold_bias = '_', '_'
            direction = row['stereo_antistereo']
            bias_type = row['bias_type']

            sent1, sent2 = '', ''
            if direction == 'stereo':
                sent1 = row['sent_more']
                sent2 = row['sent_less']
            else:
                sent1 = row['sent_less']
                sent2 = row['sent_more']

            df_item = {'sent1': sent1,
                       'sent2': sent2,
                       'direction': direction,
                       'bias_type': bias_type}
            list_data.append(df_item)
    
    df_data = pd.DataFrame.from_records(list_data)

    return df_data

In [58]:
def get_log_prob_unigram(masked_token_ids, token_ids, mask_idx, lm):
    """
    Given a sequence of token ids, with one masked token, return the log probability of the masked token.
    """
    
    model = lm["model"]
    tokenizer = lm["tokenizer"]
    log_softmax = lm["log_softmax"]
    mask_token = lm["mask_token"]
    uncased = lm["uncased"]

    # get model hidden states
    output = model(masked_token_ids)
    hidden_states = output[0].squeeze(0)
    mask_id = tokenizer.convert_tokens_to_ids(mask_token)

    # we only need log_prob for the MASK tokens
    assert masked_token_ids[0][mask_idx] == mask_id

    hs = hidden_states[mask_idx]
    target_id = token_ids[0][mask_idx]
    log_probs = log_softmax(hs)[target_id]

    return log_probs

In [59]:
def get_span(seq1, seq2):
    """
    This function extract spans that are shared between two sequences.
    """

    seq1 = [str(x) for x in seq1.tolist()]
    seq2 = [str(x) for x in seq2.tolist()]

    matcher = difflib.SequenceMatcher(None, seq1, seq2)
    template1, template2 = [], []
    for op in matcher.get_opcodes():
        # each op is a list of tuple: 
        # (operation, pro_idx_start, pro_idx_end, anti_idx_start, anti_idx_end)
        # possible operation: replace, insert, equal
        # https://docs.python.org/3/library/difflib.html
        if op[0] == 'equal':
            template1 += [x for x in range(op[1], op[2], 1)]
            template2 += [x for x in range(op[3], op[4], 1)]

    return template1, template2

In [60]:
def mask_unigram(data, lm, n=1):
    """
    Score each sentence by masking one word at a time.
    The score for a sentence is the sum of log probability of each word in
    the sentence.
    n = n-gram of token that is masked, if n > 1, we mask tokens with overlapping
    n-grams.
    """
    model = lm["model"]
    tokenizer = lm["tokenizer"]
    log_softmax = lm["log_softmax"]
    mask_token = lm["mask_token"]
    uncased = lm["uncased"]

    if torch.cuda.is_available():
        torch.set_default_tensor_type('torch.cuda.FloatTensor')

    sent1, sent2 = data["sent1"], data["sent2"]

    if uncased:
        sent1 = sent1.lower()
        sent2 = sent2.lower()

    # tokenize
    sent1_token_ids = tokenizer.encode(sent1, return_tensors='pt')
    sent2_token_ids = tokenizer.encode(sent2, return_tensors='pt')

    # get spans of non-changing tokens
    # return ids of tokens that are the same
    template1, template2 = get_span(sent1_token_ids[0], sent2_token_ids[0]) # 'equal' a[i1:i2] == b[j1:j2] (the sub-sequences are equal).

    assert len(template1) == len(template2)

    N = len(template1)  # num. of tokens that can be masked
    mask_id = tokenizer.convert_tokens_to_ids(mask_token)
    
    sent1_log_probs = 0.
    sent2_log_probs = 0.
    total_masked_tokens = 0

    # skipping CLS and SEP tokens, they'll never be masked
    for i in range(1, N-1):
        sent1_masked_token_ids = sent1_token_ids.clone().detach()
        sent2_masked_token_ids = sent2_token_ids.clone().detach()

        # mask each word 
        sent1_masked_token_ids[0][template1[i]] = mask_id
        sent2_masked_token_ids[0][template2[i]] = mask_id
        total_masked_tokens += 1

        score1 = get_log_prob_unigram(sent1_masked_token_ids, sent1_token_ids, template1[i], lm)
        score2 = get_log_prob_unigram(sent2_masked_token_ids, sent2_token_ids, template2[i], lm)

        sent1_log_probs += score1.item()
        sent2_log_probs += score2.item()

    score = {}
    # average over iterations
    score["sent1_score"] = sent1_log_probs
    score["sent2_score"] = sent2_log_probs

    return score

In [100]:
# DEBUG Purpose
# parser = argparse.ArgumentParser()
# args = parser.parse_args()
class MyDict:
    def __init__(self, data):
        self.data = data
        for key, value in data.items():
            setattr(self, key, value)

args = MyDict({
    "input_file": "crows-pairs/data/crows_pairs_anonymized.csv",
    "lm_model": "roberta",
    "output_file": "orginal_dataset/roberta/crows-pairs-output.csv"
})
# args = MyDict({
#     "input_file": "custom_dataset/CustomPrompts.csv",
#     "lm_model": "albert",
#     "output_file": f"custom_dataset/albert/CustomPrompts-output.csv"
# })

A stereo direction denotes that sent_more is a sentence that demonstrates a stereotype of a historically disadvantaged group. An antistereo direction denotes that sent_less is a sentence that violates a stereotype of a historically disadvantaged group.

In [101]:
df_data = read_data(args.input_file)

In [102]:
# df_data = df_data[df_data['bias_type'].isin(['race-color', 'gender', 'socioeconomic', 'religion'])]

In [103]:
# supported masked language models
if args.lm_model == "bert":
    tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
    model = BertForMaskedLM.from_pretrained('bert-base-uncased')
    uncased = True
elif args.lm_model == "roberta":
    tokenizer = RobertaTokenizer.from_pretrained('roberta-large')
    model = RobertaForMaskedLM.from_pretrained('roberta-large')
    uncased = False
elif args.lm_model == "albert":
    tokenizer = AlbertTokenizer.from_pretrained('albert-xxlarge-v2')
    model = AlbertForMaskedLM.from_pretrained('albert-xxlarge-v2')
    uncased = True

In [104]:
model.eval()
if torch.cuda.is_available():
    model.to('cuda')

In [105]:
mask_token = tokenizer.mask_token
log_softmax = torch.nn.LogSoftmax(dim=0)
vocab = tokenizer.get_vocab()
with open(args.lm_model + ".vocab", "w") as f:
    f.write(json.dumps(vocab))

In [106]:
lm = {"model": model,
        "tokenizer": tokenizer,
        "mask_token": mask_token,
        "log_softmax": log_softmax,
        "uncased": uncased
}

In [107]:
# score each sentence. 
# each row in the dataframe has the sentid and score for pro and anti stereo.
df_score = pd.DataFrame(columns=['sent_more', 'sent_less', 
                                    'sent_more_score', 'sent_less_score',
                                    'score', 'stereo_antistereo', 'bias_type'])

list_score = []

total_stereo, total_antistereo = 0, 0
stereo_score, antistereo_score = 0, 0

In [108]:
N = 0
neutral = 0
total = len(df_data.index)

In [109]:
with tqdm(total=total) as pbar:
    for index, data in df_data.iterrows():
        direction = data['direction']
        bias = data['bias_type']

        score = mask_unigram(data, lm)

        for stype in score.keys():
                score[stype] = round(score[stype], 3)

        N += 1
        pair_score = 0
        pbar.update(1)
        if score['sent1_score'] == score['sent2_score']:
            neutral += 1
        else:
            if direction == 'stereo':
                total_stereo += 1
                if score['sent1_score'] > score['sent2_score']:
                    stereo_score += 1
                    pair_score = 1
            elif direction == 'antistereo':
                total_antistereo += 1
                if score['sent2_score'] > score['sent1_score']:
                    antistereo_score += 1
                    pair_score = 1

        sent_more, sent_less = '', ''
        if direction == 'stereo':
            sent_more = data['sent1']
            sent_less = data['sent2']
            sent_more_score = score['sent1_score']
            sent_less_score = score['sent2_score']
        else:
            sent_more = data['sent2']
            sent_less = data['sent1']
            sent_more_score = score['sent2_score']
            sent_less_score = score['sent1_score']

        list_score.append({'sent_more': sent_more,
                                    'sent_less': sent_less,
                                    'sent_more_score': sent_more_score,
                                    'sent_less_score': sent_less_score,
                                    'score': pair_score,
                                    'stereo_antistereo': direction,
                                    'bias_type': bias
                                    })

 19%|█▊        | 280/1508 [14:54<54:38,  2.67s/it]  

In [96]:
df_score = pd.DataFrame.from_records(list_score)
df_score.to_csv(args.output_file)
print('=' * 100)
print('Total examples:', N)
print('Metric score:', round((stereo_score + antistereo_score) / N * 100, 2))
print('Stereotype score:', round(stereo_score  / total_stereo * 100, 2))
if antistereo_score != 0:
    print('Anti-stereotype score:', round(antistereo_score  / total_antistereo * 100, 2))
print("Num. neutral:", neutral, round(neutral / N * 100, 2))
print('=' * 100)

Total examples: 29
Metric score: 44.83
Stereotype score: 61.9
Num. neutral: 0 0.0


In [ ]:
# Define the statements
print_statements = [
    '=' * 100,
    f'Total examples: {N}',
    f'Metric score: {round((stereo_score + antistereo_score) / N * 100, 2)}',
    f'Stereotype score: {round(stereo_score / total_stereo * 100, 2)}'
]

# Check if there's antistereo_score before adding its line
if antistereo_score != 0:
    print_statements.append(f'Anti-stereotype score: {round(antistereo_score / total_antistereo * 100, 2)}')

print_statements.extend([
    f'Num. neutral: {neutral} {round(neutral / N * 100, 2)}',
    '=' * 100
])

# Define the file path
file_path = f"custom_dataset/{args.lm_model}/metrics.txt"

# Write to file
with open(file_path, "w") as f:
    for statement in print_statements:
        f.write(statement + '\n')